In [1]:
import qiskit

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results

from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

import qiskit.tools.jupyter

In [2]:
n_qubits = 16
n_shots = 16000

backend = Hexagonal16()
circuits = [
    state_prep_circuits.plus_state_prep(backend), 
    state_prep_circuits.equal_superposition_state_prep(backend)
           ]

## TPF
This may take a minute to compile

In [3]:
# Half shots on building, half on execution
tpf = TensorPatchFitter(backend, n_shots=n_shots / 2)
tpf.build(verbose=True)

Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0% 
	Building Edge Patches : [====================] 100.0% 
	Constructing Calibration Circuits : [====================] 100.0% 
	Executing Calibration Circuits
	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0% 
Building Patch Calibrations
	Merging Patches : [====================] 100.0% 
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0% 


## Qiskit Full
This one may take several minutes to hours to compile, and will ultimately fail anyway when being applied 
as it attempts to allocate an array of size >200Gb

In [ ]:
# Half shots on building the model, half on calibration
# For a fair test uncomment the next line, however for large n this will rapidly become useless
n_shots_qiskit_full = n_shots #// (2 ** (n_qubits - 1)) 

qr = qiskit.QuantumRegister(n_qubits)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
t_qc = qiskit.transpile(meas_calibs, backend)
cal_results = qiskit.execute(t_qc, backend, shots=n_shots_qiskit_full).result()
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
full_filter = meas_fitter.filter

## Qiskit Partial
This will compile very quickly, butwill ultimately fail anyway when being applied as it attempts to allocate an array of size >200Gb

In [45]:
# Build Calibration Matrix
mit_pattern = [[i] for i in range(n_qubits)]

# Half shots on building the model, half on calibration
n_shots_qiskit_partial = n_shots // (n_qubits / 2)

qr = qiskit.QuantumRegister(n_qubits)
meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
t_qc = qiskit.transpile(meas_calibs, backend)
cal_results = qiskit.execute(t_qc, backend, shots=n_shots).result()
meas_fitter = TensoredMeasFitter(cal_results, state_labels, circlabel='mcal')
linear_fitter = meas_fitter.filter

In [ ]:
circuit_res = []
aim_res = []
sim_res = []
tpf_res = []
for circuit in circuits:
    
    tc = qiskit.transpile(circuit,
                          backend,
                          optimization_level=0,
                          initial_layout=list(range(n_qubits))
                         )
    cr = qiskit.execute(circuit, 
                         backend, 
                         shots=n_shots / 2, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        ).result().get_counts()
    cr_n = renormalise_measurement_results(cr, 1)
    circuit_res.append(cr_n)
    
    ar = aim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True)
    ar = renormalise_measurement_results(ar, 1)
    aim_res.append(ar)
    
    sr = sim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True) 
    sr = renormalise_measurement_results(sr, 1)
    sim_res.append(sr)
    
    tr = tpf.apply(cr)
    tr = renormalise_measurement_results(tr, 1)
    tpf_res.append(tr)

In [ ]:
print(tpf_res[0]['0' * n_qubits], tpf_res[0]['1' * n_qubits])
print(aim_res[0]['0' * n_qubits], aim_res[0]['1' * n_qubits])
print(sim_res[0]['0' * n_qubits], sim_res[0]['1' * n_qubits])